# **Fine-tuning nllb-200-distilled-600M for En-Vi Machine Translation**

In [1]:
!pip install -q transformers sentencepiece datasets accelerate evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.5 MB/s eta 0:00:00


## **Dataset**

In [2]:
from datasets import load_dataset

ds = load_dataset("nguyenvuhuy/iwslt2015-en-vi")

README.md:   0%|          | 0.00/522 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/180k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/133317 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1268 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['en', 'vi'],
        num_rows: 133317
    })
    validation: Dataset({
        features: ['en', 'vi'],
        num_rows: 1268
    })
    test: Dataset({
        features: ['en', 'vi'],
        num_rows: 1268
    })
})

In [4]:
ds['train'][0]

{'en': 'Rachel Pike : The science behind a climate headline',
 'vi': 'Khoa học đằng sau một tiêu đề về khí hậu'}

## **Tokenizer**

In [5]:
from transformers import AutoTokenizer

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [6]:
len(tokenizer)

256204

## **Encoding**

In [7]:
import torchnllb-200-distilled-600M
preprocessed_ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/133317 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [8]:
preprocessed_ds['train'][0]

{'en': 'Rachel Pike : The science behind a climate headline',
 'vi': 'Khoa học đằng sau một tiêu đề về khí hậu',
 'input_ids': [256047,
  68496,
  121,
  1275,
  18816,
  1617,
  79076,
  73695,
  9,
  174670,
  32889,
  5805,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [256047,
  98,
  20194,
  10115,
  339,
  5131,
  5565,
  1777,
  22653,
  15977,
  6034,
  40243,
  80885,
  2,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -10

## **Model**

In [9]:
from transformers import AutoModelForSeq2SeqLM

model_name = "facebook/nllb-200-distilled-600M"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [10]:
model

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

## **Evaluate**

In [11]:
import numpy as np
import evaluate
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds= np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(
        preds, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )

    labels= np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(
        labels, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )

    decoded_preds, decoded_labels = postprocess_text(
        decoded_preds, decoded_labels
    )

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

## **Trainer**

In [12]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [13]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./en-vi-nllb-200_TMG301",  
    logging_strategy="no", 
    predict_with_generate=True,
    eval_strategy="steps",
    eval_steps=5000,
    save_strategy="no",  
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    run_name="en-vi-nllb-200_experiment"
)



data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model
)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=preprocessed_ds['train'],
    eval_dataset=preprocessed_ds['validation'],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu
5000,No log,1.360440,34.337886
10000,No log,1.339614,34.769544
15000,No log,1.313972,35.399828
20000,No log,1.308777,35.570811


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=24999, training_loss=1.3441811109944397, metrics={'train_runtime': 36418.6941, 'train_samples_per_second': 10.982, 'train_steps_per_second': 0.686, 'total_flos': 6.34816151894016e+16, 'train_loss': 1.3441811109944397, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub(token="")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


events.out.tfevents.1741491752.44c9184e1fe8.31.0:   0%|          | 0.00/7.12k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/32.2M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nguyenvuhuy/en-vi-nllb-200_TMG301/commit/03c592f2580f429767a4ff46f0fa3ef4d0440635', commit_message='End of training', commit_description='', oid='03c592f2580f429767a4ff46f0fa3ef4d0440635', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nguyenvuhuy/en-vi-nllb-200_TMG301', endpoint='https://huggingface.co', repo_type='model', repo_id='nguyenvuhuy/en-vi-nllb-200_TMG301'), pr_revision=None, pr_num=None)

## **Inference**

In [16]:
model_name = "nguyenvuhuy/en-vi-nllb-200_TMG301"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/39.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/32.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/4.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

### **Greedy Search**

In [17]:
src_text = "I go to school"
encoded_text = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_text
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Tôi đi học']

### **Beam search**

In [18]:
src_text = "Being a newly established university located in the business sector, FPT University has a profound understanding of market demands and is highly responsive to advancements in science and technology. From this understanding, the university has developed and implemented a distinctive training program, emphasizing individual development linked with diverse experiences and simultaneously providing updated knowledge from internationally accredited training programs along with professional skills and the capacity for lifelong self-learning to students, contributing to the development of a knowledge-based economy and the creation of a happy society."
encoded_text = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_text,
    num_beams=5,
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Là một trường đại học mới thành lập nằm trong khu vực kinh doanh trường đại học FPT có một sự hiểu biết sâu sắc về nhu cầu thị trường và đáp ứng cao với những tiến bộ trong khoa học và công nghệ . Từ sự hiểu biết này , trường đại học đã phát triển và thực hiện một chương trình đào tạo đặc biệt , nhấn mạnh sự phát triển cá nhân liên quan đến nhiều kinh nghiệm khác nhau và đồng thời cung cấp kiến thức cập nhật từ các chương trình đào']

### **Pipeline**

In [19]:
from transformers import pipeline

translator = pipeline(model="nguyenvuhuy/en-vi-nllb-200_TMG301")

Device set to use cuda:0


In [20]:
translated_text = translator("I go to school", num_beams=1, do_sample=False)
translated_text

[{'generated_text': 'Tôi đi học'}]

In [21]:
translated_text = translator("I eat fish", num_beams=2)
translated_text

[{'generated_text': 'Tôi ăn cá'}]

In [22]:
# greedy search
pred_sentences = translator(ds['test']['en'], batch_size=8, num_beams=1, do_sample=False)

In [23]:
# beam search
pred_sentences = translator(ds['test']['en'], batch_size=8, num_beams=5)

In [24]:
pred_sentences = [pred_sentence['generated_text'] for pred_sentence in pred_sentences]

In [25]:
pred_sentences[0]

'Khi còn nhỏ , tôi nghĩ đất nước mình là tốt nhất trên hành tinh , và tôi lớn lên hát một bài hát tên là &quot; Không gì đáng ghen tị . &quot;'

In [26]:
ds['test']['vi'][0]

'Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài &quot; Chúng ta chẳng có gì phải ghen tị . &quot;'

In [27]:
import sacrebleu

# greedy search
bleu_score = sacrebleu.corpus_bleu(pred_sentences, [ds['test']['vi']], force=True)
bleu_score

BLEU = 35.81 67.1/43.2/29.0/19.8 (BP = 0.996 ratio = 0.996 hyp_len = 33604 ref_len = 33738)

In [28]:
import sacrebleu

# beam search
bleu_score = sacrebleu.corpus_bleu(pred_sentences, [ds['test']['vi']], force=True)
bleu_score

BLEU = 35.81 67.1/43.2/29.0/19.8 (BP = 0.996 ratio = 0.996 hyp_len = 33604 ref_len = 33738)